In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai langchain-groq python-dotenv langchain beautifulsoup4

In [2]:
import getpass
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

## Define Models

In [3]:
# from langchain_openai import ChatOpenAI # type: ignore
# model = ChatOpenAI(model="gpt-4o-mini")

from langchain_groq import ChatGroq # type: ignore

model = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


model_with_tools = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# Agent Workflow

In [4]:
from typing import (
    Annotated,
    Sequence,
    TypedDict,
)
from langchain_core.messages import BaseMessage 
from langgraph.graph.message import add_messages 


class AgentState(TypedDict):
    """The state of the agent."""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    review_type: str
    db_type: str

## Classification Node

In [5]:
classification_prompt_template = (
                            """
                            Classify the user query into one of the following categories:  
                            - Descriptive: if the user is asking for general information or details about a particular subject.  
                            - Comparison: if the user is comparing two or more items to understand which one is better or to see differences.  
                            - Recommendation: if the user is seeking suggestions or advice for a particular choice.  

                            Based on the query, also choose the most appropriate database from the following options:  
                            - Amazon: for product-related queries or recommendations.  
                            - IMDb: for movie or TV show-related queries, comparisons, or recommendations.  
                            - Yelp: for queries related to restaurants, businesses, or local services.  

                            Examples:  
                            - Descriptive: "What is the best movie of all time?" -> Database: IMDb  
                            - Comparison: "Which movie is better, Inception or Interstellar?" -> Database: IMDb  
                            - Recommendation: "Suggest some good restaurants nearby." -> Database: Yelp  

                            Your task:  
                            Given the user query below, classify it into one of the categories (descriptive, comparison, recommendation) and choose the appropriate database (amazon, imdb, yelp).  

                            {format_instructions}  

                            Query: {query}

                            """
                        )

In [6]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import AIMessage
import json


class ClassificationNodeOutput(BaseModel):
    query: str = Field(description="The query from the user")
    query_type: str = Field(description="The type of the query: descriptive, comparison, or recommendation")
    db: str = Field(description="The type of the database to search from: amazon, imdb, or yelp")


class ClassificationNode():
    @staticmethod
    def classify_query(state: AgentState):
        parser = PydanticOutputParser(pydantic_object=ClassificationNodeOutput)
        classification_prompt = PromptTemplate(
                        template=classification_prompt_template, 
                        input_variables=["query"],
                        partial_variables={"format_instructions": parser.get_format_instructions()},
                    )
        classification_model = classification_prompt | model
        query = state["messages"][0].content
        output = classification_model.invoke({"query": query})
        response: ClassificationNodeOutput = parser.invoke(output)
        AIMessageresponse = AIMessage(content=json.dumps(response.model_dump()), name="classification_node" ,additional_kwargs = {"query": query, "json_output": response.model_dump()})
        return {"messages": [AIMessageresponse], "query_type": response.query_type, "db_type": response.db}


## Workflow

In [7]:
from langgraph.graph import StateGraph, END

def run_workflow():
    workflow = StateGraph(AgentState)

    workflow.add_node("classification node", ClassificationNode.classify_query)
    
    workflow.set_entry_point("classification node")

    workflow.add_edge("classification node", END)

    return workflow


agent_graph = run_workflow().compile(debug=True)

In [8]:
import pprint

def stream_graph(stream):
    for s in stream:
        
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            if message.name:
                if message.type == "ai":
                    print(f'================================== \033[1m{message.name} AI Message\033[0m ==================================')
                print(f"the query is: {message.additional_kwargs['query']} ")
                print(f"the string response is: {message.content}")
                print("the json response is:")
                pprint.pprint(message.additional_kwargs["json_output"])
            else:
                message.pretty_print()

In [9]:
descriptive_inputs = {"messages": [("user", "what is your opinion about the movie Avengers End Game?")]}
stream_graph(agent_graph.stream(descriptive_inputs, stream_mode="values"))

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user',
               'what is your opinion about the movie Avengers End Game?')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', 'what is your opinion about the movie Avengers End Game?')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='what is your opinion about the movie Avengers End Game?', additional_kwargs={}, response_metadata={}, id='3a2024e8-ba11-4f04-995a-3dbaf444088c')]}
[1:tasks] Starting 1 task for step 1:
- classification node -> {'messages': [HumanMessage(content='what is your opinion about the movie Avengers End Game?', additional_kwargs={}, response_metadata={}, id='3a2024e8-ba11-4f04-995a-3dbaf444088c')]}
================================ Human Message =================================

what is your opinion about the movie Avengers End Game?
[1:writes] Finished step 1 with writes 